In [1]:
import pprint
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
from datetime import datetime, tzinfo, timedelta
class simple_utc(tzinfo):
    def tzname(self,**kwargs):
        return "UTC"
    def utcoffset(self, dt):
        return timedelta(0)
datetime.utcnow().replace(tzinfo=simple_utc()).isoformat()

'2021-04-25T07:44:38.972094+00:00'

In [3]:
MONGO_USER = 'admin'
MONGO_PASSWORD = 'widmwidm9527'
MONGO_PORT = '27000'
HOST = 'localhost'
DATABASE = 'pageDB'
COLLECTION = 'autoExtractionResult'
# print(f"Loading settings, user:{MONGO_USER}, passwd:{MONGO_PASSWORD}")

MONGO_DETAILS = f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}@{HOST}:{MONGO_PORT}"

client = MongoClient(MONGO_DETAILS)

database = client[DATABASE]

page_collection = database.get_collection(COLLECTION)

In [4]:
MONGO_USER = 'admin'
MONGO_PASSWORD = 'widmwidm9527'
MONGO_PORT = '27017'
HOST = '140.115.54.44'
DATABASE = 'ETL-api-creator'
COLLECTION_urls = 'autoExtractionUrls'
COLLECTION_extractors = 'extractors'

MONGO_DETAILS_ETL = f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}@{HOST}:{MONGO_PORT}"

client_ETL = MongoClient(MONGO_DETAILS_ETL)

database_ETL = client_ETL[DATABASE]

etl_extractors_collection = database_ETL.get_collection(COLLECTION_extractors)
etl_aeUrls_collection = database_ETL.get_collection(COLLECTION_urls)

In [5]:
start_day = datetime.strptime('2021-04-22 06:14','%Y-%m-%d %H:%M')
end_day = datetime.strptime('2021-04-23 00:00','%Y-%m-%d %H:%M')

In [6]:
day_range = {
        "$gte": start_day,
        "$lt": end_day
}
print(day_range)

{'$gte': datetime.datetime(2021, 4, 22, 6, 14), '$lt': datetime.datetime(2021, 4, 23, 0, 0)}


In [7]:
Success_count = 0
Failed_count = 0
created_extractors_sns = []
for post in page_collection.find({"createdDateTime": day_range}):
    
    if post['status'] != 'Failed':
        Success_count+=1
        created_extractors_sns.append(post['serialNumber'])
    else:
#         pprint.pprint(post)
        Failed_count+=1
print("Predicted: ", Success_count+Failed_count)
print("Success: ", Success_count)
print("Failed: ", Failed_count)

Predicted:  976
Success:  842
Failed:  134


In [8]:
extractors_stat = {
    "detailPage_count": 0,
    "detailPage_success": 0,
    "detailPage_failed": 0,
    "singlePage_count": 0,
    "singlePage_success": 0,
    "singlePage_failed": 0,
}
all_succ_ext = []
for _serialNumber in created_extractors_sns:
    post = etl_extractors_collection.find_one({"serialNumber":_serialNumber})
    if post['pageType'] == 'detailPage':
        extractors_stat['detailPage_count'] +=1
        if len(post['setsColumns']) > 1:
            extractors_stat['detailPage_success'] +=1
            all_succ_ext.append(_serialNumber)
        else:
            extractors_stat['detailPage_failed'] +=1
    elif post['pageType'] == 'singleListPage':
        extractors_stat['singlePage_count'] +=1
        if len(post['setsColumns']) > 0:
            extractors_stat['singlePage_success'] +=1
            all_succ_ext.append(_serialNumber)
        else:
            extractors_stat['singlePage_failed'] +=1
            
print("Summary stats: ", extractors_stat)

Summary stats:  {'detailPage_count': 324, 'detailPage_success': 138, 'detailPage_failed': 186, 'singlePage_count': 518, 'singlePage_success': 518, 'singlePage_failed': 0}


In [9]:
single_need_crawled = 0
detailPage_need_crawled = 0
urls_count = 0
detail_succ=  set()
single_succ = set()
for _serialNumber in all_succ_ext:
    post = etl_aeUrls_collection.find_one({"serialNumber":_serialNumber})
    extractor = etl_extractors_collection.find_one({"serialNumber":_serialNumber})
    if post:
        if extractor['pageType'] == 'singleListPage':
            single_need_crawled+=1
            single_succ.add(_serialNumber)
        else:
            detailPage_need_crawled+=1
            detail_succ.add(_serialNumber)
        urls_count += len(post['urls'])
    else:
        pass
print("Single need crawled: ", single_need_crawled)
print("Detail need crawled: ", detailPage_need_crawled)
print("Urls generated: ", urls_count)

Single need crawled:  374
Detail need crawled:  132
Urls generated:  12497


In [10]:
post = etl_extractors_collection.find_one({"serialNumber":"3cvrjweyiko6tb52k"})

In [11]:
post['setsColumns']

[None]

In [12]:
len(post['setsColumns'])

1

In [151]:
len(post['setsColumns'][4])

4